In [1]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
import psycopg2
from config import db_password
import time

In [2]:
def ETL(state):
    zillow_df = pd.read_csv('Resources/Zillow_Sale_Prices_Zip.csv')
    zillow_df.set_index('RegionName', inplace=True)
    state_df = zillow_df.loc[zillow_df.StateName==state,:]
    state_df = state_df.dropna()
    median09 = state_df.loc[:,'2009-01':'2009-12'].median(axis = 1, skipna = True)
    median19 = state_df.loc[:,'2019-01':'2019-12'].median(axis = 1, skipna = True)
    new_df=pd.concat([median09, median19], axis=1)
    new_name= ['Median Price 2009','Median Price 2019']
    new_df.columns=new_name
    new_df['Inflated Median Price 2009'] = 1.1917 * new_df['Median Price 2009']
    new_df['Percent Change'] = (new_df['Median Price 2019'] - new_df['Inflated Median Price 2009'])/new_df['Inflated Median Price 2009']
    new_df = new_df[['Median Price 2009','Inflated Median Price 2009','Median Price 2019','Percent Change']]
    new_df['Median Price 2009'] = new_df['Median Price 2009'].map("${:,.2f}".format)
    new_df['Inflated Median Price 2009'] = new_df['Inflated Median Price 2009'].map("${:,.2f}".format)
    new_df['Median Price 2019'] = new_df['Median Price 2019'].map("${:,.2f}".format)
    new_df['Percent Change'] = new_df['Percent Change'].map("{:.2%}".format)
    
    return new_df.head()

In [3]:
ETL('California')

,Median Price 2009,Inflated Median Price 2009,Median Price 2019,Percent Change
RegionName,,,,
90002,"$126,300.00","$150,511.71","$390,150.00",159.22%
90016,"$275,300.00","$328,075.01","$741,600.00",126.05%
90025,"$616,700.00","$734,921.39","$947,900.00",28.98%
90032,"$239,550.00","$285,471.73","$616,650.00",116.01%
90039,"$552,300.00","$658,175.91","$1,080,600.00",64.18%


In [6]:
new_df.head()

NameError: name 'new_df' is not defined

In [7]:
def ETLCSV(state):
    zillow_df = pd.read_csv('Resources/Zillow_Sale_Prices_Zip.csv')
    zillow_df.set_index('RegionName', inplace=True)
    state_df = zillow_df.loc[zillow_df.StateName==state,:]
    state_df = state_df.dropna()
    median09 = state_df.loc[:,'2009-01':'2009-12'].median(axis = 1, skipna = True)
    median19 = state_df.loc[:,'2019-01':'2019-12'].median(axis = 1, skipna = True)
    new_df=pd.concat([median09, median19], axis=1)
    new_name= ['Median Price 2009','Median Price 2019']
    new_df.columns=new_name
    new_df['Inflated Median Price 2009'] = 1.1917 * new_df['Median Price 2009']
    new_df['Percent Change'] = (new_df['Median Price 2019'] - new_df['Inflated Median Price 2009'])/new_df['Inflated Median Price 2009']
    new_df = new_df[['Median Price 2009','Inflated Median Price 2009','Median Price 2019','Percent Change']]
    new_df['Median Price 2009'] = new_df['Median Price 2009'].map("${:,.2f}".format)
    new_df['Inflated Median Price 2009'] = new_df['Inflated Median Price 2009'].map("${:,.2f}".format)
    new_df['Median Price 2019'] = new_df['Median Price 2019'].map("${:,.2f}".format)
    new_df['Percent Change'] = new_df['Percent Change'].map("{:.2%}".format)
    
    return new_df.to_csv ('california_df.csv', index = False, header=True)

In [8]:
ETLCSV('California')